## Trabajo 1: Solución lineal de sistemas lineales en Julia

* Objetivo:Comparar diferentes métodos de resolución de sistemas lineales de gran tamaño con un problema de EDP que genera matrices no simétricas en base a los parámetros N, α y ε=1

$$\beta \cdot \bigtriangledown u-\epsilon \bigtriangleup u=0 $$

$$ \beta =\alpha (\frac{\sqrt{2}}{2},\frac{\sqrt{2}}{2})^T $$

Contenido: utilizar el código en python (basado en el código Matlab de A. Meister, Aufgabe 6) y las librerías Julia de C.T. Kelley en https://github.com/ctkelley

Metodología: realizar un notebook de jupyter en Julia con gráficas y tablas de errores y tiempos de cómputo en base a las iteraciones

In [31]:
using LinearAlgebra
using SparseArrays

# Parameters
a = 0.5;   # Advection parameter
b = 2;     # Reaction parameter
tol = 1e-6;    # Tolerance for stopping criterion
maxit = 10000;  # Maximum number of iterations

# Linear system
N = 10;
dx = 1 / (N + 1);

e = ones(N);
Dxx = spdiagm(-1 => e[1:N-1] , 0 => -2 * e  , 1 => e[1:N-1]);   # 1D central difference

ident = Matrix{Float64}(I, N, N);
x = kron(ident, Dxx) / (dx^2);  # 2nd derivative with respect to x

Axx = kron(ident, Dxx) / (dx^2); # 2nd derivative with respect to x
Ayy = kron(Dxx, ident) / (dx^2);  # 2nd derivative with respect to y

Dx = spdiagm( -1 => e[1:N-1] , 1 => e[1:N-1]);  # 1D central difference
Ax = kron(ident, Dx) / (2 * dx);

I_2 = Matrix{Float64}(I, N*N, N*N);

A = -Axx - Ayy - a * Ax - b * I_2;  # Matrix
b = ones(N * N);  # Right-hand side
u0 = 0.5 * ones(N * N);  # Initial guess

1. Utilizar métodos directo LU y Choleski para matrices simétricas, y los métodos iterativos CG, BiCG , BiCGStab , GMRES , LGMRES, MINRES, QMR que encuentren en las librerías de Julia. Si encuentran otros , los aportan

Metodos directos:

a. LU

In [32]:
using LinearAlgebra

#Calcular descomposicion LU
lu_decomposition = lu(A);
L = lu_decomposition.L;
U = lu_decomposition.U;
P = lu_decomposition.p;

b. Choleski

In [33]:
using LinearAlgebra

chol_decomposition=cholesky(A);
L = chol_decomposition.L; 

Metodos iterativos:

CG:

In [45]:
using IterativeSolvers
x=cg(A,b);
println("solucion:");
x

solucion:


100-element Vector{Float64}:
 0.015132176191275281
 0.02573483662399068
 0.03312296138180865
 0.03784904595323942
 0.040162168491703884
 0.04016216849170386
 0.03784904595323944
 0.03312296138180864
 0.025734836623990676
 0.015132176191275288
 ⋮
 0.025734836623990665
 0.03312296138180864
 0.03784904595323943
 0.04016216849170388
 0.04016216849170388
 0.037849045953239416
 0.03312296138180865
 0.02573483662399067
 0.015132176191275281

BiCG:

In [35]:
using IterativeSolvers
#x0=zeros(length(b));
#bicg(A,b)

BiCGStab

In [36]:
using IterativeSolvers
x=bicgstabl(A,b);

GMRES

In [37]:
using IterativeSolvers
x=gmres(A,b);

LGMRES

MINRES

QMR

2. Utilizar escenarios de N y α en base a la presentación . Utilizar el mayor N possible con tiempos esperables menos de una hora).

3. Calcular tiempos y errores en base a las iteraciones de los métodos iterativos y comparer con los métodos directos

4. Analizar los resultados y decidir el métodos más rápido y preciso para cada escenario de N y αα.

5. Encontrar las propiedades de las matrices generadas por la linearización en redes neuronales y proponer qué método sería el major para problemas de alta dimensionalidad .